In [3]:
import csv
from uncertainties import ufloat

# Cargar datos del archivo CSV
data = []
with open('../../Downloads/mag_err.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Convertir las columnas numéricas a float
        row['mag_g'] = float(row['mag_g']); row['sigma_g'] = float(row['sigma_g'])
        row['mag_i'] = float(row['mag_i']); row['sigma_i'] = float(row['sigma_i'])
        row['E(B-V)'] = float(row['E(B-V)']); row['E(B-V)_ERR'] = float(row['E(B-V)_ERR'])
        data.append(row)

# Ejemplo: crear ufloat para el primer objeto (ID=1) y mostrar sus valores
obj1 = data[0]
mg1 = ufloat(obj1['mag_g'], obj1['sigma_g'])   # magnitud g con error
mi1 = ufloat(obj1['mag_i'], obj1['sigma_i'])   # magnitud i con error
EBV1 = ufloat(obj1['E(B-V)'], obj1['E(B-V)_ERR'])  # E(B-V) con error
print(f"Objeto ID1 - m_g = {mg1}, m_i = {mi1}, E(B-V) = {EBV1}")


Objeto ID1 - m_g = 16.76+/-0.08, m_i = 17.06+/-0.09, E(B-V) = 0.109+/-0.013


In [4]:
import uncertainties.umath as umath  # funciones matemáticas compatibles con ufloat

# Constante R_V de Calzetti
Rv = 4.05

# Longitudes de onda en micrones
lambda_g = 4770.8 / 1e4   # 4770.8 Å = 0.47708 μm
lambda_i = 7774.2 / 1e4   # 7774.2 Å = 0.77742 μm

# Calcular k(lambda) para cada filtro usando la ley de Calzetti
if lambda_g < 0.63:
    k_g = 2.659 * (-2.156 + 1.509/lambda_g - 0.198/(lambda_g**2) + 0.011/(lambda_g**3)) + Rv
if lambda_i >= 0.63:
    k_i = 2.659 * (-1.857 + 1.040/lambda_i) + Rv

print(f"k'(λ_g = 4770.8 Å) = {k_g:.3f}")
print(f"k'(λ_i = 7774.2 Å) = {k_i:.3f}")


k'(λ_g = 4770.8 Å) = 4.684
k'(λ_i = 7774.2 Å) = 2.669


In [7]:
# Recorrer cada objeto y calcular A_g, A_i, m_g0, m_i0 con incertidumbres
results = []
for obj in data:
    mg = ufloat(obj['mag_g'], obj['sigma_g'])
    mi = ufloat(obj['mag_i'], obj['sigma_i'])
    EBV = ufloat(obj['E(B-V)'], obj['E(B-V)_ERR'])
    # Extinción en cada filtro
    A_g = k_g * EBV
    A_i = k_i * EBV
    # Magnitudes corregidas por extinción
    m_g0 = mg - A_g
    m_i0 = mi - A_i
    # Guardar resultados (podemos almacenar ufloat o sus nominales y desviaciones)
    results.append({
        'ID': obj['ID'],
        'A_g': A_g, 'A_i': A_i,
        'm_g0': m_g0, 'm_i0': m_i0
    })
    # Imprimir resultados para este objeto
    print(f"ID {obj['ID']}: A_g = {A_g:.3f}, A_i = {A_i:.3f}, "
          f"m_g0 = {m_g0:.3f}, m_i0 = {m_i0:.3f}")


ID 1: A_g = 0.511+/-0.061, A_i = 0.291+/-0.035, m_g0 = 16.250+/-0.098, m_i0 = 16.765+/-0.095
ID 2: A_g = 0.253+/-0.075, A_i = 0.144+/-0.043, m_g0 = 17.115+/-0.127, m_i0 = 17.122+/-0.106
ID 3: A_g = 0.501+/-0.080, A_i = 0.286+/-0.045, m_g0 = 16.762+/-0.125, m_i0 = 16.758+/-0.099
ID 4: A_g = 0.206+/-0.061, A_i = 0.117+/-0.035, m_g0 = 16.895+/-0.109, m_i0 = 17.139+/-0.103
ID 5: A_g = 0.590+/-0.075, A_i = 0.336+/-0.043, m_g0 = 16.416+/-0.114, m_i0 = 16.878+/-0.104
ID 6: A_g = 0.131+/-0.070, A_i = 0.075+/-0.040, m_g0 = 16.975+/-0.115, m_i0 = 17.423+/-0.115
ID 7: A_g = 0.253+/-0.061, A_i = 0.144+/-0.035, m_g0 = 18.670+/-0.218, m_i0 = 18.852+/-0.219
ID 8: A_g = 0.276+/-0.089, A_i = 0.157+/-0.051, m_g0 = 17.782+/-0.166, m_i0 = 17.890+/-0.149
ID 9: A_g = 0.829+/-0.094, A_i = 0.472+/-0.053, m_g0 = 16.762+/-0.147, m_i0 = 16.783+/-0.111
ID 10: A_g = 0.155+/-0.066, A_i = 0.088+/-0.037, m_g0 = 17.391+/-0.129, m_i0 = 17.130+/-0.102
ID 11: A_g = 0.187+/-0.075, A_i = 0.107+/-0.043, m_g0 = 18.794+/-0.22

In [10]:
import math
from uncertainties import ufloat

# --- Módulo de distancia (distancia fija: 4.19 Mpc) ---
distance_pc = 4.19e6  # 4.19 Mpc en parsec
print(distance_pc)

# --- Magnitud absoluta en i: M_i = m_i0 - 5*log10(distance_pc) + 5 ---
for res in results:
    M_i = res['m_i0'] - 5.0 * math.log10(distance_pc) + 5.0  # ufloat - float + float
    res['M_i'] = M_i

    # Imprimir con mayor precisión
    print(f"ID {res['ID']}: M_i = {M_i.nominal_value:.6f} ± {M_i.std_dev:.6f}")


4190000.0
ID 1: M_i = -11.346028 ± 0.094595
ID 2: M_i = -10.989214 ± 0.105986
ID 3: M_i = -11.352689 ± 0.099011
ID 4: M_i = -10.972521 ± 0.103020
ID 5: M_i = -11.233406 ± 0.104159
ID 6: M_i = -10.687812 ± 0.115183
ID 7: M_i = -9.259214 ± 0.218770
ID 8: M_i = -10.221561 ± 0.148904
ID 9: M_i = -11.328543 ± 0.110721
ID 10: M_i = -10.981158 ± 0.102086
ID 11: M_i = -9.311844 ± 0.211360
ID 12: M_i = -8.942222 ± 0.254174
ID 13: M_i = -9.118094 ± 0.226480
ID 14: M_i = -8.122102 ± 0.351934
ID 15: M_i = -9.395972 ± 0.215269
ID 16: M_i = -8.914302 ± 0.271045
ID 17: M_i = -9.593779 ± 0.192133
ID 19: M_i = -8.065425 ± 0.388545
ID 20: M_i = -7.974844 ± 0.469193


In [17]:
# --- Masa estelar estimada con la ecuación (8) de Taylor+2011 ---
for res in results:
    # Color g-i con corrección por extinción
    color_gi = res['m_g0'] - res['m_i0']
    # log10(M*/M☉) según Taylor+2011 (ecuación 8)
    logM = 1.15 + 0.70 * color_gi - 0.4 * res['M_i']
    res['logM'] = logM
    # Masa en unidades de M☉ (propagando la incertidumbre)
    M_star = 10**logM
    res['M_star'] = M_star
    # Imprimir resultado con su incertidumbre
    print(f"ID {res['ID']}: log10(M/M\u2609) = {logM.nominal_value:.6f} ± {logM.std_dev:.6f}")


ID 1: log10(M/M☉) = 5.328206 ± 0.110884
ID 2: log10(M/M☉) = 5.540938 ± 0.128502
ID 3: log10(M/M☉) = 5.693491 ± 0.118383
ID 4: log10(M/M☉) = 5.368462 ± 0.123991
ID 5: log10(M/M☉) = 5.320085 ± 0.120724
ID 6: log10(M/M☉) = 5.111241 ± 0.134898
ID 7: log10(M/M☉) = 4.726438 ± 0.279065
ID 8: log10(M/M☉) = 5.163126 ± 0.182654
ID 9: log10(M/M☉) = 5.667023 ± 0.132998
ID 10: log10(M/M☉) = 5.725529 ± 0.130309
ID 11: log10(M/M☉) = 4.870832 ± 0.272997
ID 12: log10(M/M☉) = 4.638111 ± 0.326929
ID 13: log10(M/M☉) = 4.638946 ± 0.277085
ID 14: log10(M/M☉) = 4.871219 ± 0.507650
ID 15: log10(M/M☉) = 4.577097 ± 0.266787
ID 16: log10(M/M☉) = 4.527239 ± 0.340233
ID 17: log10(M/M☉) = 5.186850 ± 0.242058
ID 19: log10(M/M☉) = 4.380289 ± 0.463524
ID 20: log10(M/M☉) = 4.182032 ± 0.492716


In [19]:
import csv

# Suponiendo que ya existen las listas 'data' (filas originales) y 'results' (resultados calculados) de celdas previas

# Combinar columnas originales con las nuevas columnas calculadas
filas_salida = []
for obj_orig, obj_calc in zip(data, results):
    fila = {
        # columnas originales
        'ID': obj_orig['ID'],
        'mag_g': obj_orig['mag_g'],
        'sigma_g': obj_orig['sigma_g'],
        'mag_i': obj_orig['mag_i'],
        'sigma_i': obj_orig['sigma_i'],
        'E(B-V)': obj_orig['E(B-V)'],
        'E(B-V)_ERR': obj_orig['E(B-V)_ERR'],
        # nuevas columnas calculadas (valor nominal e incertidumbre)
        'A_g': obj_calc['A_g'].nominal_value,        # extinción banda g
        'sigma_A_g': obj_calc['A_g'].std_dev,        # incertidumbre de A_g
        'A_i': obj_calc['A_i'].nominal_value,        # extinción banda i
        'sigma_A_i': obj_calc['A_i'].std_dev,        # incertidumbre de A_i
        'm_g0': obj_calc['m_g0'].nominal_value,      # magnitud g corregida
        'sigma_m_g0': obj_calc['m_g0'].std_dev,      # incertidumbre de m_g0
        'm_i0': obj_calc['m_i0'].nominal_value,      # magnitud i corregida
        'sigma_m_i0': obj_calc['m_i0'].std_dev,      # incertidumbre de m_i0
        'M_i': obj_calc['M_i'].nominal_value,        # magnitud absoluta i
        'sigma_M_i': obj_calc['M_i'].std_dev,        # incertidumbre de M_i
        'logM': obj_calc['logM'].nominal_value,      # log10(M/M☉)
        'sigma_logM': obj_calc['logM'].std_dev,      # incertidumbre de logM
        'M_star': obj_calc['M_star'].nominal_value,  # masa estelar en M☉
        'sigma_M_star': obj_calc['M_star'].std_dev   # incertidumbre de M_star
    }
    filas_salida.append(fila)

# Especificar las columnas en el orden deseado (opcional, para asegurar el orden en el CSV)
columnas = list(filas_salida[0].keys())

# Escribir a un nuevo archivo CSV en la ruta deseada (misma carpeta que el original)
ruta_salida = '../../Downloads/mag_err_resultados.csv'  # puedes cambiar nombre si lo deseas
with open(ruta_salida, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=columnas)
    writer.writeheader()
    for fila in filas_salida:
        writer.writerow(fila)

print(f"Archivo CSV guardado en {ruta_salida} con {len(columnas)} columnas (originales + nuevas).")


Archivo CSV guardado en ../../Downloads/mag_err_resultados.csv con 21 columnas (originales + nuevas).
